In [3]:
import pandas as pd
import scipy.stats
import numpy as np, random
import datetime as dt
import matplotlib.pyplot as plt
import io

from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [194]:
filename = "/Users/pengyang/Desktop/Fall 2025/Capstone/UCLA_Microsoft_Data.xlsx"

ContosoRevData = pd.read_excel(filename, sheet_name=1)
FortuneGlobal2000 = pd.read_excel(filename, sheet_name=2)
TAM = pd.read_excel(filename, sheet_name=3)

## **0.Necessary Feature Engineering**

1. Adding Geo & Category market share

In [197]:
# Step 0: Filter for TAM data of Product A to E (exclude RevX)
product_tam = TAM[TAM['Metric_Name'].str.contains("Proj_TAM_Prod")].copy()

# Step 1: Aggregate TAM by Geo_Entity and Commercial_Category
geo_tam = product_tam[product_tam['Attr_Name'] == 'Geo_Entity'] \
    .groupby('Attr_Value')['Metric_Value'].sum().reset_index() \
    .rename(columns={'Attr_Value': 'Geo_Entity', 'Metric_Value': 'Geo_TAM_FY30'})

cat_tam = product_tam[product_tam['Attr_Name'] == 'Commercial_Category'] \
    .groupby('Attr_Value')['Metric_Value'].sum().reset_index() \
    .rename(columns={'Attr_Value': 'Commercial_Category', 'Metric_Value': 'Category_TAM_FY30'})

# Step 2: Merge aggregated TAM into ContosoRevData
ContosoRevData = ContosoRevData.merge(geo_tam, on='Geo_Entity', how='left')
ContosoRevData = ContosoRevData.merge(cat_tam, on='Commercial_Category', how='left')

# Step 3: Calculate FY30 Market Share
ContosoRevData['MarketShare_FY30_Geo'] = ContosoRevData['PotentialRevenue_FY30'] / ContosoRevData['Geo_TAM_FY30']
ContosoRevData['MarketShare_FY30_Category'] = ContosoRevData['PotentialRevenue_FY30'] / ContosoRevData['Category_TAM_FY30']



2. Adding FY22-25 revenue growth CAGR

In [199]:
ContosoRevData['Revenue_CAGR'] = (
    (ContosoRevData['TotalRevenue_FY25'] / ContosoRevData['TotalRevenue_FY22']) ** (1 / 3) - 1) * 100

3. Adding FY25 to FY30 revenue potential

In [201]:
ContosoRevData['Revenue_potential'] = ContosoRevData['PotentialRevenue_FY30'] - ContosoRevData['TotalRevenue_FY25']

4. Adding Ordinal Market Tier and Tier Change

In [203]:
# Define mapping: Tier A is best (1), Tier D is lowest (4)
tier_map = {
    'Tier A': 1,
    'Tier B': 2,
    'Tier C': 3,
    'Tier D': 4
}

# List all your tier columns
tier_cols = ['MarketTier_FY22', 'MarketTier_FY23', 'MarketTier_FY24', 'MarketTier_FY25', 'MarketTier_FY26']

# Apply ordinal encoding
for col in tier_cols:
    ContosoRevData[col + '_ordinal'] = ContosoRevData[col].map(tier_map)

# Quick check
ContosoRevData[[col for col in ContosoRevData.columns if 'MarketTier' in col]].head()

,MarketTier_FY26,MarketTier_FY25,MarketTier_FY24,MarketTier_FY23,MarketTier_FY22,MarketTier_FY22_ordinal,MarketTier_FY23_ordinal,MarketTier_FY24_ordinal,MarketTier_FY25_ordinal,MarketTier_FY26_ordinal
0,Tier B,Tier B,Tier B,Tier B,Tier B,2,2,2,2,2
1,Tier D,Tier D,Tier D,Tier D,Tier D,4,4,4,4,4
2,Tier B,Tier B,Tier B,Tier B,Tier D,4,2,2,2,2
3,Tier B,Tier B,Tier B,Tier B,Tier B,2,2,2,2,2
4,Tier D,Tier D,Tier D,Tier D,Tier D,4,4,4,4,4


5. Adding YoY Rev/Tier Changes

Since Tier_Change_FY25_FY26 ranges from -2 to 2 but tier can only be adjusted by 1, we modify the column to only have {-1,0,1} to represent downgrade, stay and upgrade.

In [206]:
# Loop through FY22 to FY25 to compute year-over-year changes
for year in range(22, 26):  # FY22 to FY25
    prev = f'FY{year}'
    curr = f'FY{year + 1}'

    # Tier Change: positive = downgrade, negative = upgrade
    ContosoRevData[f'Tier_Change_{prev}_{curr}'] = (
        ContosoRevData[f'MarketTier_{curr}_ordinal'] - ContosoRevData[f'MarketTier_{prev}_ordinal']
    )
    ContosoRevData[f'Tier_Change_{prev}_{curr}']=[min(x,1) if x >=0 else -1 for x in ContosoRevData[f'Tier_Change_{prev}_{curr}']]
    # Revenue Change: absolute change
    ContosoRevData[f'Revenue_Change_{prev}_{curr}'] = (
        ContosoRevData[f'TotalRevenue_{curr}'] - ContosoRevData[f'TotalRevenue_{prev}']
    )

    # Revenue Growth: % change (must come AFTER revenue change is created!)
    ContosoRevData[f'Revenue_Growth_{prev}_{curr}'] = (
        ContosoRevData[f'Revenue_Change_{prev}_{curr}'] / ContosoRevData[f'TotalRevenue_{prev}']
    )

In [207]:
ContosoRevData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 53 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         50000 non-null  int64  
 1   Geo_Entity                 50000 non-null  object 
 2   Area_National              50000 non-null  object 
 3   Area_Admin                 19100 non-null  object 
 4   Area_Urban                 50000 non-null  object 
 5   Commercial_Category        50000 non-null  object 
 6   Commercial_Stream          50000 non-null  object 
 7   Commercial_Specialty       50000 non-null  object 
 8   MarketDomain               50000 non-null  object 
 9   MarketTier_FY26            50000 non-null  object 
 10  MarketTier_FY25            50000 non-null  object 
 11  MarketTier_FY24            50000 non-null  object 
 12  MarketTier_FY23            50000 non-null  object 
 13  MarketTier_FY22            50000 non-null  obj

## **1. Modeling Setup**

#### 1.0 Set Seed
We set a random seed at the start of the modeling section to ensure reproducibility.
Although Ward clustering itself is deterministic, other steps like PCA, K-Means initialization, or data sampling involve randomness. Fixing a seed makes sure that results remain consistent every time the notebook is re-run.

In [217]:
np.random.seed(42)
random.seed(42)

#### Train / Test Split
Since our goal is to use historial data to predict future tier dynamics, columns of year 2026 are removed.

In [222]:
# =========================================================
# 1. Define FY26 columns to drop from X
# =========================================================
fy26_cols = [
    'RevStreamX_FY26',
    'ProdCatA_Revenue_FY26',
    'ProdCatB_Revenue_FY26',
    'ProdCatC_Revenue_FY26',
    'ProdCatD_Revenue_FY26',
    'ProdCatE_Revenue_FY26',
    'TotalRevenue_FY26',
    'MarketTier_FY26',   # remove from X but keep for y
    'MarketTier_FY26_ordinal',
    'Tier_Change_FY25_FY26',
    'Revenue_Change_FY25_FY26',
    'Revenue_Growth_FY25_FY26'
]
# =========================================================
# 2. Drop all letter market tier and ID columns
# =========================================================
drop_cols = [
    'ID',
    'Area_Admin',
    'MarketTier_FY25',
    'MarketTier_FY24',
    'MarketTier_FY23',
    'MarketTier_FY22',
]
X = ContosoRevData.drop(columns=drop_cols)
# =========================================================
# 3. X = all columns except FY26 columns
# =========================================================
X = X.drop(columns=fy26_cols)

# =========================================================
# 4. y = ordinal MarketTier_FY26 (already encoded)
# =========================================================
y = ContosoRevData['Tier_Change_FY25_FY26']


# =========================================================
# 5. Train/Test Split (70/30)
# =========================================================
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.30,
    random_state=42
)

# =========================================================
# 6. Print results
# =========================================================
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test :", X_test.shape)
print("y_test :", y_test.shape)

print("\nUnique encoded tiers:", sorted(ContosoRevData['MarketTier_FY26'].unique()))


X_train: (35000, 35)
y_train: (35000,)
X_test : (15000, 35)
y_test : (15000,)

Unique encoded tiers: ['Tier A', 'Tier B', 'Tier C', 'Tier D']


In [224]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35000 entries, 38094 to 15795
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Geo_Entity                 35000 non-null  object 
 1   Area_National              35000 non-null  object 
 2   Area_Urban                 35000 non-null  object 
 3   Commercial_Category        35000 non-null  object 
 4   Commercial_Stream          35000 non-null  object 
 5   Commercial_Specialty       35000 non-null  object 
 6   MarketDomain               35000 non-null  object 
 7   TotalRevenue_FY25          35000 non-null  float64
 8   TotalRevenue_FY24          35000 non-null  float64
 9   TotalRevenue_FY23          35000 non-null  int64  
 10  TotalRevenue_FY22          35000 non-null  int64  
 11  RevStreamX_FY25            35000 non-null  int64  
 12  RevStreamX_FY24            35000 non-null  int64  
 13  RevStreamX_FY23            35000 non-null  int6

In [226]:
y_train.unique()

array([ 0,  1, -1])

## 2. Model Training

### 2.1 Multinomial Logistic Model

In [63]:
# =========================================================
# Build training dataframe for statsmodels
# =========================================================
train_df = X_train.copy()
train_df["Tier_Change_FY25_FY26"] = y_train

test_df = X_test.copy()
test_df["Tier_Change_FY25_FY26"] = y_test


In [64]:
# =========================================================
# Identify numeric and categorical features
# =========================================================
numeric_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
categorical_cols = X_train.select_dtypes(exclude=["number"]).columns.tolist()

print("Numeric columns:", numeric_cols)
print("\nCategorical columns:", categorical_cols)


Numeric columns: ['TotalRevenue_FY25', 'TotalRevenue_FY24', 'TotalRevenue_FY23', 'TotalRevenue_FY22', 'RevStreamX_FY25', 'RevStreamX_FY24', 'RevStreamX_FY23', 'RevStreamX_FY22', 'PotentialRevenue_FY30', 'Geo_TAM_FY30', 'Category_TAM_FY30', 'MarketShare_FY30_Geo', 'MarketShare_FY30_Category', 'Revenue_CAGR', 'Revenue_potential', 'MarketTier_FY22_ordinal', 'MarketTier_FY23_ordinal', 'MarketTier_FY24_ordinal', 'MarketTier_FY25_ordinal', 'Tier_Change_FY22_FY23', 'Revenue_Change_FY22_FY23', 'Revenue_Growth_FY22_FY23', 'Tier_Change_FY23_FY24', 'Revenue_Change_FY23_FY24', 'Revenue_Growth_FY23_FY24', 'Tier_Change_FY24_FY25', 'Revenue_Change_FY24_FY25', 'Revenue_Growth_FY24_FY25']

Categorical columns: ['Geo_Entity', 'Area_National', 'Area_Admin', 'Area_Urban', 'Commercial_Category', 'Commercial_Stream', 'Commercial_Specialty', 'MarketDomain']


In [65]:
# =========================================================
# Build formula with ALL features
# =========================================================

formula_parts = []

# add numeric features
for col in numeric_cols:
    formula_parts.append(col)

# add categorical features (wrapped with C())
for col in categorical_cols:
    formula_parts.append(f"C({col})")

# Join into formula
formula_full = "Tier_Change_FY25_FY26 ~ " + " + ".join(formula_parts)

print("FULL MODEL FORMULA:\n", formula_full)


FULL MODEL FORMULA:
 Tier_Change_FY25_FY26 ~ TotalRevenue_FY25 + TotalRevenue_FY24 + TotalRevenue_FY23 + TotalRevenue_FY22 + RevStreamX_FY25 + RevStreamX_FY24 + RevStreamX_FY23 + RevStreamX_FY22 + PotentialRevenue_FY30 + Geo_TAM_FY30 + Category_TAM_FY30 + MarketShare_FY30_Geo + MarketShare_FY30_Category + Revenue_CAGR + Revenue_potential + MarketTier_FY22_ordinal + MarketTier_FY23_ordinal + MarketTier_FY24_ordinal + MarketTier_FY25_ordinal + Tier_Change_FY22_FY23 + Revenue_Change_FY22_FY23 + Revenue_Growth_FY22_FY23 + Tier_Change_FY23_FY24 + Revenue_Change_FY23_FY24 + Revenue_Growth_FY23_FY24 + Tier_Change_FY24_FY25 + Revenue_Change_FY24_FY25 + Revenue_Growth_FY24_FY25 + C(Geo_Entity) + C(Area_National) + C(Area_Admin) + C(Area_Urban) + C(Commercial_Category) + C(Commercial_Stream) + C(Commercial_Specialty) + C(MarketDomain)


In [89]:
def fit_and_eval_mnlogit(cols):
    """
    Fit a Multinomial Logit (MNLogit) model using the selected columns `cols`,
    and evaluate its accuracy and classification metrics on the test set.
    """

    # 1) Select subset of features
    Xtr = X_train[cols].copy()
    Xte = X_test[cols].copy()
    ytr = y_train.copy()
    yte = y_test.copy()

    # 2) Standardize features (important to avoid overflow in MNLogit exp() step)
    scaler = StandardScaler()
    Xtr_scaled = scaler.fit_transform(Xtr)
    Xte_scaled = scaler.transform(Xte)

    # 3) Convert back to DataFrame (preserve column names and index)
    Xtr_scaled_df = pd.DataFrame(Xtr_scaled, columns=cols, index=Xtr.index)
    Xte_scaled_df = pd.DataFrame(Xte_scaled, columns=cols, index=Xte.index)

    # 4) Add constant term required by statsmodels MNLogit
    Xtr_sm = sm.add_constant(Xtr_scaled_df)
    Xte_sm = sm.add_constant(Xte_scaled_df)

    # 5) Fit MNLogit model
    model = sm.MNLogit(ytr, Xtr_sm)
    res = model.fit(method="newton", maxiter=200, full_output=True, disp=False)

    # 6) Predict probabilities for each class
    #    (column order = sorted(unique_classes))
    probs = res.predict(Xte_sm)         # shape: (n_samples, n_classes)
    probs_arr = np.asarray(probs)

    # 7) Convert internal index → actual labels (-1, 0, 1)
    unique_classes = np.sort(ytr.unique())      # e.g., array([-1, 0, 1])
    pred_idx = probs_arr.argmax(axis=1)         # index of max probability
    y_pred = pd.Series(unique_classes[pred_idx], index=Xte.index)

    y_pred = y_pred.astype(yte.dtype)

    # 8) Print evaluation results
    acc = accuracy_score(yte, y_pred)
    print("========================================")
    print("Features used:", cols)
    print("Accuracy:", acc)
    print("\nClassification Report:\n", classification_report(yte, y_pred))
    print("\nConfusion Matrix:\n", confusion_matrix(yte, y_pred))
    print("========================================\n")

    return res, acc


In [94]:
def summarize_var_importance_mnlogit(res):
    """
    For a statsmodels MNLogit result object:
    - Flatten coef & p-values into a long table (class × variable)
    - Aggregate by variable to compute min/mean p-values and max |coef|
    """

    params = res.params   # rows: variables, cols: classes (non-baseline)
    pvals = res.pvalues

    rows = []

    # params is a DataFrame: index = variable names, columns = class labels
    for var in params.index:
        for cls in params.columns:
            coef_val = params.loc[var, cls]
            pval_val = pvals.loc[var, cls]

            rows.append({
                "class": str(cls),
                "variable": var,
                "coef": float(coef_val),
                "pval": float(pval_val)
            })

    coef_long = pd.DataFrame(rows)

    # Drop intercept
    coef_long = coef_long[coef_long["variable"] != "const"].copy()

    # Safety: ensure numeric dtype
    coef_long["coef"] = pd.to_numeric(coef_long["coef"], errors="coerce")
    coef_long["pval"] = pd.to_numeric(coef_long["pval"], errors="coerce")

    # Aggregate per variable
    var_summary = (
        coef_long
        .groupby("variable")
        .agg(
            min_pval=("pval", "min"),
            mean_pval=("pval", "mean"),
            max_abs_coef=("coef", lambda s: s.abs().max())
        )
        .reset_index()
    )

    # Mark whether variable is significant at 5% level for at least one class
    var_summary["is_sig_0_05"] = var_summary["min_pval"] < 0.05

    # Sort: significant first, then by min_pval ascending
    var_summary = var_summary.sort_values(
        ["is_sig_0_05", "min_pval"],
        ascending=[False, True]
    )

    return coef_long, var_summary


In [95]:
def backward_select_by_pvalue(initial_cols, p_thresh=0.10, min_vars=5):
    """
    Backward selection for MNLogit based on variable-level min p-values.

    Parameters
    ----------
    initial_cols : list of str
        Starting feature set.
    p_thresh : float
        Threshold for min_pval above which a variable can be dropped.
    min_vars : int
        Minimum number of variables to keep.

    Returns
    -------
    selected_cols : list of str
        Final selected features.
    final_res : MNLogit result object
        Fitted result on the final selected features.
    final_acc : float
        Accuracy on the test set for the final model.
    final_var_summary : DataFrame
        Variable summary (min_pval, mean_pval, max_abs_coef, is_sig_0_05).
    """

    cols = initial_cols.copy()
    iteration = 0

    while True:
        iteration += 1
        print(f"\n=== Backward Selection Iteration {iteration} ===")
        print("Current number of features:", len(cols))
        print("Current feature set:", cols)

        # Fit model on current feature set
        res, acc = fit_and_eval_mnlogit(cols)

        # Summarize variable importance
        coef_long, var_summary = summarize_var_importance_mnlogit(res)
        print("\nTop variables by significance (smallest min_pval):")
        print(var_summary.head(10))

        # Find the least significant variable (largest min_pval)
        worst = var_summary.sort_values("min_pval", ascending=False).iloc[0]
        worst_var = worst["variable"]
        worst_p = worst["min_pval"]

        print(f"\nLeast significant variable: {worst_var} (min_pval = {worst_p:.4f})")

        # Stopping conditions
        if (worst_p <= p_thresh) or (len(cols) <= min_vars):
            print("\nStopping backward selection.")
            print(f"Reason: {'p-value threshold reached' if worst_p <= p_thresh else 'min_vars reached'}")
            final_res = res
            final_acc = acc
            final_var_summary = var_summary
            break

        # Drop the least significant variable and continue
        print(f"Dropping variable: {worst_var}")
        cols.remove(worst_var)

    return cols, final_res, final_acc, final_var_summary


In [96]:
selected_cols, final_res, final_acc, final_var_summary = backward_select_by_pvalue(
    numeric_cols,
    p_thresh=0.10,   # you can tighten to 0.05 later
    min_vars=6       # don't go below 6 variables
)

print("\n=== FINAL SELECTED FEATURES ===")
print(selected_cols)
print("\nFinal model accuracy:", final_acc)

print("\n=== FINAL VARIABLE SUMMARY (by p-value) ===")
print(final_var_summary)



=== Backward Selection Iteration 1 ===
Current number of features: 28
Current feature set: ['TotalRevenue_FY25', 'TotalRevenue_FY24', 'TotalRevenue_FY23', 'TotalRevenue_FY22', 'RevStreamX_FY25', 'RevStreamX_FY24', 'RevStreamX_FY23', 'RevStreamX_FY22', 'PotentialRevenue_FY30', 'Geo_TAM_FY30', 'Category_TAM_FY30', 'MarketShare_FY30_Geo', 'MarketShare_FY30_Category', 'Revenue_CAGR', 'Revenue_potential', 'MarketTier_FY22_ordinal', 'MarketTier_FY23_ordinal', 'MarketTier_FY24_ordinal', 'MarketTier_FY25_ordinal', 'Tier_Change_FY22_FY23', 'Revenue_Change_FY22_FY23', 'Revenue_Growth_FY22_FY23', 'Tier_Change_FY23_FY24', 'Revenue_Change_FY23_FY24', 'Revenue_Growth_FY23_FY24', 'Tier_Change_FY24_FY25', 'Revenue_Change_FY24_FY25', 'Revenue_Growth_FY24_FY25']
Features used: ['TotalRevenue_FY25', 'TotalRevenue_FY24', 'TotalRevenue_FY23', 'TotalRevenue_FY22', 'RevStreamX_FY25', 'RevStreamX_FY24', 'RevStreamX_FY23', 'RevStreamX_FY22', 'PotentialRevenue_FY30', 'Geo_TAM_FY30', 'Category_TAM_FY30', 'Marke

In [97]:
core_vars = final_var_summary[
    (final_var_summary["is_sig_0_05"]) &
    (final_var_summary["max_abs_coef"] > 0.3)
]["variable"].tolist()

core_vars

['TotalRevenue_FY23',
 'Revenue_CAGR',
 'Revenue_Growth_FY22_FY23',
 'Tier_Change_FY23_FY24',
 'MarketTier_FY25_ordinal',
 'MarketTier_FY23_ordinal',
 'Revenue_Change_FY24_FY25',
 'Revenue_Growth_FY23_FY24',
 'Tier_Change_FY24_FY25',
 'MarketTier_FY22_ordinal',
 'Tier_Change_FY22_FY23',
 'Revenue_Growth_FY24_FY25',
 'Revenue_Change_FY23_FY24',
 'Revenue_Change_FY22_FY23',
 'RevStreamX_FY25',
 'RevStreamX_FY23']

In [98]:
res_core, acc_core = fit_and_eval_mnlogit(core_vars)
print("Core model variables:", core_vars)
print("Core model accuracy:", acc_core)


Features used: ['TotalRevenue_FY23', 'Revenue_CAGR', 'Revenue_Growth_FY22_FY23', 'Tier_Change_FY23_FY24', 'MarketTier_FY25_ordinal', 'MarketTier_FY23_ordinal', 'Revenue_Change_FY24_FY25', 'Revenue_Growth_FY23_FY24', 'Tier_Change_FY24_FY25', 'MarketTier_FY22_ordinal', 'Tier_Change_FY22_FY23', 'Revenue_Growth_FY24_FY25', 'Revenue_Change_FY23_FY24', 'Revenue_Change_FY22_FY23', 'RevStreamX_FY25', 'RevStreamX_FY23']
Accuracy: 0.9464

Classification Report:
               precision    recall  f1-score   support

          -1       0.76      0.04      0.07       420
           0       0.95      1.00      0.97     14192
           1       0.39      0.03      0.06       388

    accuracy                           0.95     15000
   macro avg       0.70      0.36      0.37     15000
weighted avg       0.93      0.95      0.92     15000


Confusion Matrix:
 [[   16   404     0]
 [    5 14167    20]
 [    0   375    13]]

Core model variables: ['TotalRevenue_FY23', 'Revenue_CAGR', 'Revenue_Growth_F

**Summary** <br>
- Built a full statistical pipeline (scaling, log-transform, strict p-value elimination), reducing **28** original features down to **21** statistically meaningful drivers.
- These retained variables mainly capture revenue-growth dynamics and historical tier momentum, showing clear links to transition likelihood.
- Model accuracy reaches **~94%**, but real tier movements are extremely rare (only **808 / 15,000 = 5.4%** upgraded/downgraded), so the model predicts most of clients as “no change.”

Multinomial logit model fails not because of coding issues, but because **the model class is structurally incompatible with our data.** Tier transitions are extremely rare — only **808 of 15,000 cases (5.4%)** moved tiers, while **95%** stayed unchanged. MNLogit maximizes overall likelihood, so under such imbalance it collapses almost all predictions into the majority “no-change” class. As a linear-boundary GLM, it cannot capture the weak, nonlinear signals driving rare tier movements. This makes MNLogit unsuitable for forecasting transitions and justifies exploring alternative modeling approaches.

### 2.2 Neural Networks
#### 2.2.1 Data Encoding and Standardization

In [163]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

## Encoding Categorical Variables
drop_cols=['Geo_Entity','Area_Urban','Commercial_Specialty','Area_Admin']
df=X.copy()
df=df.drop(columns=drop_cols)
cat_cols=['Area_National','Commercial_Category','Commercial_Stream','MarketDomain']
num_cols=[x for x in df.columns if x not in cat_cols]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(drop="first",handle_unknown='ignore',
            sparse_output=False), cat_cols)
    ]
)
X_std= preprocess.fit_transform(df)
X_train_std, X_test_std, y_train, y_test = train_test_split(
    X_std,
    y,
    test_size=0.30,
    random_state=42
)

In [165]:
X_train_std.shape

(35000, 218)

#### 2.2.2 Neural Network with Original Dataset

In [168]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    hidden_layer_sizes=(10,5),
    activation="relu",
    solver="adam",
    max_iter=500,
    random_state=42
)

mlp.fit(X_train_std, y_train)
y_pred = mlp.predict(X_test_std)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.70      0.48      0.57       420
           0       0.97      0.98      0.98     14192
           1       0.45      0.27      0.34       388

    accuracy                           0.95     15000
   macro avg       0.71      0.58      0.63     15000
weighted avg       0.94      0.95      0.95     15000



In [170]:
## Confusion Matrix of Test Data
confusion_matrix(y_test, y_pred)

array([[  202,   218,     0],
       [   86, 13978,   128],
       [    0,   282,   106]])

In [172]:
## Training Data Performance
print(classification_report(y_train, mlp.predict(X_train_std)))

              precision    recall  f1-score   support

          -1       0.87      0.58      0.69      1028
           0       0.97      0.99      0.98     32943
           1       0.74      0.40      0.52      1029

    accuracy                           0.96     35000
   macro avg       0.86      0.66      0.73     35000
weighted avg       0.96      0.96      0.96     35000



#### 2.2.3 Neural Network with Oversampled Data

In [175]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)

X_res, y_res = ros.fit_resample(X_train_std, y_train)

mlp2 = MLPClassifier(
    hidden_layer_sizes=(10,5),
    activation="relu",
    solver="adam",
    max_iter=1000,
    random_state=42
)
mlp2.fit(X_res, y_res)
y_pred = mlp2.predict(X_test_std)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.16      0.67      0.26       420
           0       0.98      0.76      0.86     14192
           1       0.13      0.77      0.23       388

    accuracy                           0.76     15000
   macro avg       0.43      0.73      0.45     15000
weighted avg       0.94      0.76      0.83     15000



**Summary** <br>
The model with imbalanced data is very good at identifying companies that stay in the same tier, but it struggles to detect upgrades and downgrades. Roughly half of true downgrades and almost 80% of true upgrades are wrongly classified. So while overall accuracy is high, the model is currently weak at finding the companies where something actually changes, which are often the most important from a business perspective.

Oversampling helped improve the recall of upgraded companies but accuracy and precision dorpped a lot.

### 2.3 CatBoost Model

In [180]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score, precision_score
)

from catboost import CatBoostClassifier, Pool
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import xgboost as xgb

# Toggle any block ON/OFF
RUN_CATBOOST_BASE        = True
RUN_CATBOOST_WEIGHTED    = True
RUN_CATBOOST_OVERSAMPLE  = True
RUN_XGB_BASE             = True
RUN_XGB_WEIGHTED         = True

In [229]:
X_train0 = X_train.copy()
X_test0  = X_test.copy()
y_train0 = y_train.copy()
y_test0  = y_test.copy()

categorical_cols = [
    'Geo_Entity', 'Area_National', 'Area_Admin', 'Area_Urban',
    'Commercial_Category', 'Commercial_Stream',
    'Commercial_Specialty', 'MarketDomain'
]

classes_original = np.array([-1, 0, 1])

#### Setting up Utility Functions

In [232]:
# =========================================================
# BLOCK 2 — Utility Functions (shared everywhere)
# =========================================================
def prepare_catboost_data(X_train, X_test, y_train, y_test, categorical_cols):
    X_train = X_train.copy()
    X_test  = X_test.copy()

    # Fill NA & cast categoricals to str
    for col in categorical_cols:
        if col in X_train.columns:
            X_train[col] = X_train[col].astype(str).fillna("Unknown")
            X_test[col]  = X_test[col].astype(str).fillna("Unknown")

    cat_features_idx = [
        X_train.columns.get_loc(c)
        for c in categorical_cols
        if c in X_train.columns
    ]

    train_pool = Pool(X_train, y_train, cat_features=cat_features_idx)
    test_pool  = Pool(X_test,  y_test,  cat_features=cat_features_idx)

    return X_train, X_test, train_pool, test_pool, cat_features_idx


def print_full_metrics(y_train, train_pred, y_test, test_pred, title="MODEL"):
    print(f"\n==================== {title} ====================")
    print("Train Accuracy:", accuracy_score(y_train, train_pred))
    print("Test Accuracy :", accuracy_score(y_test,  test_pred))
    print("\nClassification Report:\n",
          classification_report(y_test, test_pred, zero_division=0))
    print("\nConfusion Matrix:\n", confusion_matrix(y_test, test_pred))

    print("\nMacro F1:", f1_score(y_test, test_pred, average='macro'))
    print("Weighted F1:", f1_score(y_test, test_pred, average='weighted'))

    print("\nPrediction Distribution:", np.unique(test_pred, return_counts=True))


def label_encode_for_xgb(X_train, X_test, categorical_cols):
    X_train = X_train.copy()
    X_test  = X_test.copy()
    encoders = {}

    for col in categorical_cols:
        if col in X_train.columns:
            le = LabelEncoder()
            X_train[col] = le.fit_transform(X_train[col].astype(str))

            X_test[col] = X_test[col].astype(str)
            X_test[col] = X_test[col].apply(lambda x: x if x in le.classes_ else "Unknown")

            if "Unknown" not in le.classes_:
                le.classes_ = np.append(le.classes_, "Unknown")

            X_test[col] = le.transform(X_test[col])

            encoders[col] = le

    return X_train, X_test, encoders


def remap_classes_for_xgb(y_train, y_test):
    class_map   = {-1: 0, 0: 1, 1: 2}
    inverse_map = {v: k for k, v in class_map.items()}
    return y_train.map(class_map), y_test.map(class_map), inverse_map


#### 2.3.1 Catboost Baseline Model

In [235]:
# =========================================================
# BLOCK 3 — CatBoost BASELINE
# =========================================================
if RUN_CATBOOST_BASE:

    Xtr, Xte, train_pool, test_pool, cat_idx = prepare_catboost_data(
        X_train0, X_test0, y_train0, y_test0, categorical_cols
    )

    model = CatBoostClassifier(
        iterations=700, learning_rate=0.04, depth=6,
        loss_function='MultiClass', eval_metric='Accuracy',
        random_state=42, verbose=100
    )

    model.fit(train_pool, eval_set=test_pool, use_best_model=True)

    train_pred = model.predict(train_pool).astype(int).flatten()
    test_pred  = model.predict(test_pool).astype(int).flatten()

    print_full_metrics(y_train0, train_pred, y_test0, test_pred, "CatBoost BASELINE")

0:	learn: 0.9412857	test: 0.9460667	best: 0.9460667 (0)	total: 29.7ms	remaining: 20.7s
100:	learn: 0.9633143	test: 0.9668667	best: 0.9668667 (99)	total: 2.48s	remaining: 14.7s
200:	learn: 0.9673143	test: 0.9700000	best: 0.9700000 (197)	total: 5.27s	remaining: 13.1s
300:	learn: 0.9685143	test: 0.9709333	best: 0.9710000 (283)	total: 8.14s	remaining: 10.8s
400:	learn: 0.9694286	test: 0.9712667	best: 0.9713333 (392)	total: 11.1s	remaining: 8.3s
500:	learn: 0.9700857	test: 0.9713333	best: 0.9714667 (424)	total: 14.1s	remaining: 5.6s
600:	learn: 0.9706571	test: 0.9713333	best: 0.9715333 (515)	total: 17.2s	remaining: 2.83s
699:	learn: 0.9712857	test: 0.9714000	best: 0.9715333 (515)	total: 20.1s	remaining: 0us

bestTest = 0.9715333333
bestIteration = 515

Shrink model to first 516 iterations.

==================== CatBoost BASELINE ====================
Train Accuracy: 0.97
Test Accuracy : 0.9715333333333334

Classification Report:
               precision    recall  f1-score   support

       

#### 2.3.2 CatBoost Model with Balanced Class Weights

In [237]:
# =========================================================
# BLOCK 4 — CatBoost BALANCED CLASS WEIGHTS
# =========================================================
if RUN_CATBOOST_WEIGHTED:

    Xtr, Xte, train_pool, test_pool, cat_idx = prepare_catboost_data(
        X_train0, X_test0, y_train0, y_test0, categorical_cols
    )

    class_weights_arr = compute_class_weight(
        class_weight='balanced', classes=classes_original, y=y_train0
    )
    class_weights = {c: w for c, w in zip(classes_original, class_weights_arr)}

    model = CatBoostClassifier(
        iterations=800, learning_rate=0.04, depth=6,
        loss_function='MultiClass', eval_metric='Accuracy',
        class_weights=class_weights,
        random_state=42, verbose=100
    )

    model.fit(train_pool, eval_set=test_pool, use_best_model=True)

    train_pred = model.predict(train_pool).astype(int).flatten()
    test_pred  = model.predict(test_pool).astype(int).flatten()

    print_full_metrics(y_train0, train_pred, y_test0, test_pred, "CatBoost BALANCED WEIGHTS")

0:	learn: 0.7851117	test: 0.7732279	best: 0.7732279 (0)	total: 29.3ms	remaining: 23.4s
100:	learn: 0.8320912	test: 0.8129079	best: 0.8129079 (100)	total: 2.79s	remaining: 19.3s
200:	learn: 0.8496236	test: 0.8213766	best: 0.8233253 (192)	total: 5.68s	remaining: 16.9s
300:	learn: 0.8726469	test: 0.8247470	best: 0.8255966 (296)	total: 8.48s	remaining: 14.1s
400:	learn: 0.8939472	test: 0.8258443	best: 0.8262697 (394)	total: 11.3s	remaining: 11.3s
500:	learn: 0.9086918	test: 0.8210163	best: 0.8275405 (431)	total: 14.2s	remaining: 8.47s
600:	learn: 0.9206813	test: 0.8180415	best: 0.8275405 (431)	total: 17.1s	remaining: 5.65s
700:	learn: 0.9323977	test: 0.8147650	best: 0.8275405 (431)	total: 19.8s	remaining: 2.8s
799:	learn: 0.9410754	test: 0.8126896	best: 0.8275405 (431)	total: 22.5s	remaining: 0us

bestTest = 0.8275404519
bestIteration = 431

Shrink model to first 432 iterations.

==================== CatBoost BALANCED WEIGHTS ====================
Train Accuracy: 0.8177428571428571
Test Acc

#### 2.3.3 CatBoost Model with Oversampling

In [241]:
# =========================================================
# BLOCK 5 — CatBoost OVERSAMPLE + STRONG WEIGHTS
# =========================================================
if RUN_CATBOOST_OVERSAMPLE:

    Xtr, Xte, train_pool_tmp, test_pool_tmp, cat_idx = prepare_catboost_data(
        X_train0, X_test0, y_train0, y_test0, categorical_cols
    )

    # Oversample -1 and +1
    minority_idx = y_train0[y_train0 != 0].index
    X_min = Xtr.loc[minority_idx]
    y_min = y_train0.loc[minority_idx]

    X_over = pd.concat([Xtr, X_min.sample(8000, replace=True)])
    y_over = pd.concat([y_train0, y_min.sample(8000, replace=True)])

    strong_weights = {-1: 30, 0: 1, 1: 30}

    train_pool = Pool(X_over, y_over, cat_features=cat_idx)
    test_pool  = Pool(Xte, y_test0,   cat_features=cat_idx)

    model = CatBoostClassifier(
        iterations=1500, learning_rate=0.02, depth=8,
        loss_function='MultiClass', eval_metric='Accuracy',
        class_weights=strong_weights,
        random_state=42, verbose=100
    )

    model.fit(train_pool, eval_set=test_pool, use_best_model=True)

    train_pred = model.predict(train_pool).astype(int).flatten()
    test_pred  = model.predict(test_pool).astype(int).flatten()

    print_full_metrics(y_over, train_pred, y_test0, test_pred,
                       "CatBoost OVERSAMPLED + STRONG WEIGHTS")


0:	learn: 0.5224606	test: 0.5495420	best: 0.5495420 (0)	total: 23.1ms	remaining: 34.6s
100:	learn: 0.6228631	test: 0.3873595	best: 0.7434169 (47)	total: 3.86s	remaining: 53.5s
200:	learn: 0.6340598	test: 0.3896493	best: 0.7434169 (47)	total: 8.06s	remaining: 52.1s
300:	learn: 0.6451309	test: 0.3891028	best: 0.7434169 (47)	total: 12.6s	remaining: 50.1s
400:	learn: 0.6556075	test: 0.3889207	best: 0.7434169 (47)	total: 16.9s	remaining: 46.4s
500:	learn: 0.6684865	test: 0.3871253	best: 0.7434169 (47)	total: 21.7s	remaining: 43.4s
600:	learn: 0.6886506	test: 0.3859544	best: 0.7434169 (47)	total: 27s	remaining: 40.4s
700:	learn: 0.7133150	test: 0.3866049	best: 0.7434169 (47)	total: 32.2s	remaining: 36.7s
800:	learn: 0.7409317	test: 0.3877238	best: 0.7434169 (47)	total: 37.4s	remaining: 32.7s
900:	learn: 0.7649057	test: 0.3878799	best: 0.7434169 (47)	total: 42.6s	remaining: 28.3s
1000:	learn: 0.7868628	test: 0.3897273	best: 0.7434169 (47)	total: 47.7s	remaining: 23.8s
1100:	learn: 0.8060289	t

**Summary** <br>
CatBoost delivers strong overall performance with high accuracy **(0.97)** and a solid macro F1 score **(0.76)**, driven largely by its excellent ability to identify customers who remain in the same tier, where it achieves perfect recall and strong precision. However, despite strong performance on the majority class, CatBoost still struggles to detect true tier movements: it captures only **58%** of downgrades and **44%** of upgrades, meaning more than half of actual transitions are missed and absorbed into the “no change” prediction. This indicates that while CatBoost is highly reliable for stability detection, its ability to identify the minority cases—customers whose tiers are shifting—is still limited, which poses challenges from a business perspective where timely detection of these transitions is most important.

### 2.4 XGBoost Model
#### 2.4.1 XGBoost Baseline Model

In [259]:
# =========================================================
# BLOCK 6 — XGBoost BASELINE
# =========================================================
if RUN_XGB_BASE:

    Xtr, Xte, enc = label_encode_for_xgb(X_train0, X_test0, categorical_cols)
    ytr_m, yte_m, invmap = remap_classes_for_xgb(y_train0, y_test0)

    model = XGBClassifier(
        objective='multi:softmax', num_class=3,
        eval_metric='mlogloss',
        learning_rate=0.04, max_depth=7, n_estimators=700,
        subsample=0.9, colsample_bytree=0.9,
        random_state=42, tree_method='hist'
    )

    model.fit(Xtr, ytr_m)

    train_pred = pd.Series(model.predict(Xtr)).map(invmap)
    test_pred  = pd.Series(model.predict(Xte)).map(invmap)

    print_full_metrics(y_train0, train_pred, y_test0, test_pred, "XGBoost BASELINE")



==================== XGBoost BASELINE ====================
Train Accuracy: 0.9998285714285714
Test Accuracy : 0.9711333333333333

Classification Report:
               precision    recall  f1-score   support

          -1       0.98      0.58      0.73       420
           0       0.97      1.00      0.98     14192
           1       0.83      0.44      0.58       388

    accuracy                           0.97     15000
   macro avg       0.93      0.67      0.76     15000
weighted avg       0.97      0.97      0.97     15000


Confusion Matrix:
 [[  243   177     0]
 [    6 14152    34]
 [    0   216   172]]

Macro F1: 0.7635047651498063
Weighted F1: 0.9671981260222683

Prediction Distribution: (array([-1,  0,  1]), array([  249, 14545,   206]))


#### 2.4.2 XGBoost Model with Balanced Class Weights

In [250]:
# =========================================================
# BLOCK 7 — XGBoost WEIGHTED
# =========================================================
if RUN_XGB_WEIGHTED:

    Xtr, Xte, enc = label_encode_for_xgb(X_train0, X_test0, categorical_cols)
    ytr_m, yte_m, invmap = remap_classes_for_xgb(y_train0, y_test0)

    class_counts = ytr_m.value_counts().to_dict()
    total = len(ytr_m)
    sample_weights = ytr_m.map(lambda c: total / class_counts[c])

    model = XGBClassifier(
        objective='multi:softmax', num_class=3,
        eval_metric='mlogloss',
        learning_rate=0.03, max_depth=7, n_estimators=700,
        subsample=0.9, colsample_bytree=0.9,
        random_state=42, tree_method='hist'
    )

    model.fit(Xtr, ytr_m, sample_weight=sample_weights)

    train_pred = pd.Series(model.predict(Xtr)).map(invmap)
    test_pred  = pd.Series(model.predict(Xte)).map(invmap)

    print_full_metrics(y_train0, train_pred, y_test0, test_pred, "XGBoost WEIGHTED")



==================== XGBoost WEIGHTED ====================
Train Accuracy: 0.9922857142857143
Test Accuracy : 0.9540666666666666

Classification Report:
               precision    recall  f1-score   support

          -1       0.83      0.61      0.70       420
           0       0.98      0.97      0.98     14192
           1       0.42      0.59      0.49       388

    accuracy                           0.95     15000
   macro avg       0.74      0.72      0.72     15000
weighted avg       0.96      0.95      0.96     15000


Confusion Matrix:
 [[  255   164     1]
 [   52 13828   312]
 [    0   160   228]]

Macro F1: 0.7226967431199139
Weighted F1: 0.9555066646794005

Prediction Distribution: (array([-1,  0,  1]), array([  307, 14152,   541]))


**Summary** <br>
XGBoost shows nearly identical performance to CatBoost, achieving high accuracy **(0.97)** and a macro F1 score of **0.76**, with particularly strong results on the dominant “no change” class, where it reaches perfect recall and high precision. Like CatBoost, XGBoost struggles with the rare but critical tier transitions, detecting only **58%** of downgrades and **44%** of upgrades, leaving over half of true changes unrecognized. This imbalance-driven limitation means that, although XGBoost is stable and effective at modeling overall tier behavior, it remains less effective at surfacing the minority class events that matter most for proactive account management and strategic decision-making.